In [33]:
import pandas as pd
import numpy as np
import scipy.misc
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
import tensorflow as tf
import ron_gauss

In [34]:
df = pd.read_csv('ground_truth_new.csv')

In [ ]:
len(df)

In [35]:
df.columns

Index(['taxi_id', 'shift', 'company_id', 'pickup_community_area',
       'dropoff_community_area', 'payment_type', 'trip_day_of_week',
       'trip_hour_of_day', 'fare', 'tips', 'trip_total', 'trip_seconds',
       'trip_miles'],
      dtype='object')

In [36]:
drop = ['taxi_id', 'company_id']
categorical = ["trip_day_of_week", "payment_type", "dropoff_community_area", "pickup_community_area", "shift"]
continuous = [x for x in df.columns if x not in (categorical + drop)]
print(continuous)

['trip_hour_of_day', 'fare', 'tips', 'trip_total', 'trip_seconds', 'trip_miles']


In [46]:
def supervised_rongauss(df, dim, target):
    target_y = df[target].to_numpy()
    df = df.drop(columns=[target])

    y_max = target_y.max()
    df = df.to_numpy()
    rongauss_unsup = ron_gauss.RONGauss(algorithm='supervised')
    dp_x, dp_y = rongauss_unsup.generate_dpdata(df, dim, target_y, y_max)
    return dp_x, dp_y

In [47]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [49]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def baseline_model():
    model = Sequential()
    model.add(Dense(33, input_dim=12, activation='relu'))
    model.add(Dense(33, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#Continuous variables
from sklearn import preprocessing
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

features = set(df.columns)
for target_feature in continuous:
    print(target_feature)
    input_features = features - {target_feature}
    input_data = df[input_features]
    target = df[target_feature]
    ron_gauss_data, private_target = supervised_rongauss(df=df, dim=10, target=target_feature)
    
    X_train, X_test, y_train, y_test = train_test_split(ron_gauss_data, private_target, random_state=42)

    pipe = make_pipeline(StandardScaler(), Normalizer(), KerasRegressor(build_fn=baseline_model, epochs=150, batch_size=10, verbose=0))

    print("Starting fit...")
    pipe.fit(X_train, y_train)

    start_time = time.time()
    accuracy = pipe.score(X_test, y_test)

    accuracy_dataframe[target_feature] = accuracy
    print("(time={}s) Regression + RON_Gauss, target_feature: {}, prediction_avg_accuracy: {}".format(time.time() - start_time, target_feature, accuracy))


trip_hour_of_day
Starting fit...
